In [1]:
import pandas as pd
import math
import numpy as np
from datetime import datetime
from IPython.display import display,HTML
date = "2024-05-22"

In [2]:
df = pd.read_excel('conversation_test_2205_.xlsx')
df.head(4)

,created_at,updated_at,closed_at,source_type,source_name,user_id,user_replies_count,user_reply_in_average,user_reply_in_average_bh,user_reply_in_average_count
0,2024-05-22 07:14:37,2024-05-22 09:33:44,2024-05-22 09:22:42,message réseau socio X,message de l'intreprise X,100000001ABCDE,8,8986,1786,5
1,2024-05-22 07:16:03,2024-05-22 09:51:40,2024-05-22 09:51:38,message réseau socio X,message de l'intreprise X,100000002ABCDE,3,44052,8052,1
2,2024-05-22 07:23:14,2024-05-22 10:01:00,2024-05-22 10:00:59,message réseau socio X,message de l'intreprise X,100000002ABCDE,6,10794,1794,4
3,2024-05-22 07:23:52,2024-05-22 11:41:55,2024-05-22 11:41:50,message réseau socio X,message de l'intreprise X,100000002ABCDE,7,8669,1469,5


In [3]:
import pandas as pd
from IPython.display import display, HTML
df['data_closed_at'] = pd.to_datetime(df['closed_at'].astype(str).str.split().str[0])
df[['user_reply_in_average_bh', 'user_reply_in_average_count']] = df[['user_reply_in_average_bh', 'user_reply_in_average_count']].fillna(0)
df['on_time_list'] = df['user_reply_in_average_bh'] * df['user_reply_in_average_count']
nombre_conversation = df['data_closed_at'].eq(date).sum()
nombre_conversation_inf_10800 = df.query('on_time_list <= 10800 and data_closed_at == @date').shape[0]
OnTime = f"{(nombre_conversation_inf_10800 / nombre_conversation * 100):.2f}%"
back_log = df.query('on_time_list > 10800 and data_closed_at == @date')['on_time_list'].mean()
back_log = (back_log - 10800) / 10800 if pd.notna(back_log) else 0

style = "<style>.result {text-transform:uppercase;color:rgb(255,102,0);font-weight:bold;text-align:center;padding:20px;font-size:20px;margin:auto;border:2px solid}</style>"
display(HTML(style + f"<div class='result'><span>On Time: </span>{OnTime}</div>"))
display(HTML(style + f"<div class='result'><span>Back Log: </span>{back_log}</div>"))


In [4]:
message = pd.read_excel('message_test_2205_.xlsx')
date_message = "2024-05-22"
message.head(5)

,created_at,updated_at,source_type,source_name,type,status
0,2024-05-22 00:01:01,2024-05-22 08:39:07,Page du réseaux socio X,Réseau socio de l'entreprise X,Commentaire,Archivé
1,2024-05-22 00:02:48,2024-05-22 08:39:07,Page du réseaux socio X,Réseau socio de l'entreprise X,Commentaire,Archivé
2,2024-05-22 00:03:09,2024-05-22 08:39:07,Page du réseaux socio X,Réseau socio de l'entreprise X,Commentaire,Archivé
3,2024-05-22 00:19:37,2024-05-22 08:39:07,Page du réseaux socio X,Réseau socio de l'entreprise X,Commentaire,Archivé
4,2024-05-22 00:29:33,2024-05-22 08:39:07,Page du réseaux socio X,Réseau socio de l'entreprise X,Commentaire,Archivé


In [5]:
def transformation(message1,nom_colonne) :
    message_301 = message1[nom_colonne].value_counts().sort_index()
    df_intervalles1 = pd.DataFrame(message_301)
    df_intervalles1 = df_intervalles1.reset_index()
    df_intervalles1['index'] = pd.to_datetime(df_intervalles1['index'], format='%H:%M:%S').apply(lambda x: datetime(2000, 1, 1, x.hour, x.minute, x.second))
    index_range1 = pd.date_range(start='2000-01-01 00:00:00', end='2000-01-01 23:30:00', freq='30T')
    df_intervalles1 = df_intervalles1.set_index('index').reindex(index_range1, fill_value=0).reset_index()
    df_intervalles1['tranche_30min'] = df_intervalles1['index'].dt.time
    df_intervalles1 = df_intervalles1.drop(columns={'index'})
    df_intervalles1 = df_intervalles1.set_index('tranche_30min')
    return df_intervalles1

message['nb_massage_reçu'] = message['created_at'].dt.floor('30T').dt.time
message1 = message[(message['status']=='Répondu') | (message['status']=='En traitement') | (message['status']=='Nouveau') | (message['status']=='Archivé')]

message['nb_message_traité'] = message['created_at'].dt.floor('30T').dt.time
message2 = message[(message['status']=='Répondu') | (message['status']=='En traitement') | (message['status']=='Archivé')]

message['nb_message_sortant'] = message['created_at'].dt.floor('30T').dt.time
message3 = message[(message['status']=='Réponse agent') | (message['status']=='Message agent')]

df_intervalles1 = transformation(message1,'nb_massage_reçu')
df_intervalles2 = transformation(message2,'nb_message_traité')
df_intervalles3 = transformation(message3,'nb_message_sortant')

resultat = pd.concat([df_intervalles1,df_intervalles2,df_intervalles3],axis=1)
resultat

,nb_massage_reçu,nb_message_traité,nb_message_sortant
tranche_30min,,,
00:00:00,5,5,0
00:30:00,2,2,0
01:00:00,1,1,0
01:30:00,1,1,0
02:00:00,2,2,0
02:30:00,1,1,0
03:00:00,1,1,0
03:30:00,2,2,0
04:00:00,3,3,0
